# Aula 25 – Rastreamento

## 1. Introdução

Até agora estudamos apenas imagens paradas, mas também podemos tratar a visão computacional em vídeos. Uma das tarefas que podemos realizar é o rastreamento, que permite seguir um objeto em vídeo. Para realizar essa tarefa, fazemos o cálculo do Fluxo Óptico, isto é, as variações nas direções de intensidades dos pixels na imagem com o passar do tempo.

Mais especificamente, um algoritmo simples e interessante para fluxo óptico e, consequentemente, rastreamento, é chamado de Lucas-Kanade. Nele, calcula-se o fluxo de uma janela de pixels predeterminada (e não da imagem toda). Dados, então, esses pontos para serem “seguidos”, o algoritmo calcula as derivadas parciais, ou seja, variação da intensidade dos pixels, nos eixos x, y e no tempo. Com isso, obtém-se um sistema de equações que dará o vetor de deslocamento do centro da janela. Observe que ele não acompanha os pixels da janela, mas sim a variação da intensidade na vizinhança, e tenta descobrir qual deveria ser a nova posição do ponto com tal informação. Mais detalhes sobre a matemática e como implementá-lo, veja a leitura complementar.

Observe que para realizar o rastreamento com Lucas-Kanade, primeiro é necessário, na imagem inicial, identificar o objeto que você gostaria de rastrear. Isso pode ser feito de forma automática, como veremos em aulas futuras, ou pode ser manualmente. Essa identificação marcará a janela de pixels que precisa ser passada para o algoritmo.

Outro ponto importante é que o Lucas-Kanade assume que em um intervalo de tempo pequeno, existe relativamente pouca movimentação dos pixels e, portanto, é possível acompanhar a direção da variação de intensidade na imagem. Tal presunção, entretanto, não é verdade para objetos muito rápidos e o Lucas-Kanade tende a perdê-los (“rápido”, entretanto, é relativo à taxa de quadros do vídeo, que se for muito baixa pode atrapalhar inclusive o rastreamento de objetos lentos). Além disso, como o algoritmo não “procura” pontos em quadros subsequentes, apenas estima a nova posição pela variação da intensidade dos pixels de uma janela analisada, ele acaba sendo bastante sensível a ruído, calculando novas posições que não são de fato as posições do objeto de interesse no novo quadro.

## 2. Leitura Complementar

http://www.inf.fu-berlin.de/inst/ag-ki/rojas_home/documents/tutorials/Lucas-Kanade2.pdf

## 3. Exercícios

A OpenCV possui o algoritmo de Lucas-Kanade implementado. Escreva um programa simples que abra um vídeo, permita ao usuário marcar alguns pontos com o mouse e utilize o Lucas-Kanade para rastrear esses pontos.

In [4]:
import cv2
import numpy as np
 
cap = cv2.VideoCapture(0)
ret, frame1 = cap.read()
prvs = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[...,1] = 255
 
while(1):
    ret, frame2 = cap.read()
    next = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)

    flow = cv2.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
    hsv[...,0] = ang*180/np.pi/2
    hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    rgb = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)

    cv2.imshow('frame2',rgb)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    elif k == ord('s'):
        cv2.imwrite('opticalfb.png',frame2)
        cv2.imwrite('opticalhsv.png',rgb)
    prvs = next
   
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
 
cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.4.0) /tmp/pip-req-build-6179nsls/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
